<a href="https://colab.research.google.com/github/pankajti/wqu/blob/main/capstone/module4/breakout_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [39]:
## import list of all NSE tickers 

In [40]:
import pandas as pd 
import yfinance as yf 

In [41]:
import datetime as dt

In [42]:
start = (dt.datetime.now()-dt.timedelta(6) ).strftime('%Y-%m-%d')

In [43]:
# list of all companies from https://www.nseindia.com/regulations/listing-compliance/nse-market-capitalisation-all-companies

In [44]:
tickers_path = '/Users/pankajti/dev/git/wqu/capstone/module4/MCAP31122023_0.xlsx'
tickers_data = pd.read_excel(tickers_path,nrows=2189)

In [45]:
yahoo_symbs=tickers_data.Symbol+'.NS'

In [46]:
yahoo_symbs= yahoo_symbs.dropna().tolist()

In [47]:
sum(pd.isna(yahoo_symbs))

0

In [48]:
len(yahoo_symbs)

2189

In [49]:
yahoo_symbs

['RELIANCE.NS',
 'TCS.NS',
 'HDFCBANK.NS',
 'ICICIBANK.NS',
 'INFY.NS',
 'HINDUNILVR.NS',
 'BHARTIARTL.NS',
 'ITC.NS',
 'SBIN.NS',
 'LICI.NS',
 'LT.NS',
 'BAJFINANCE.NS',
 'HCLTECH.NS',
 'KOTAKBANK.NS',
 'AXISBANK.NS',
 'ASIANPAINT.NS',
 'TITAN.NS',
 'ADANIENT.NS',
 'MARUTI.NS',
 'ULTRACEMCO.NS',
 'SUNPHARMA.NS',
 'NTPC.NS',
 'BAJAJFINSV.NS',
 'DMART.NS',
 'TATAMOTORS.NS',
 'ONGC.NS',
 'NESTLEIND.NS',
 'ADANIGREEN.NS',
 'WIPRO.NS',
 'COALINDIA.NS',
 'ADANIPORTS.NS',
 'POWERGRID.NS',
 'JSWSTEEL.NS',
 'M&M.NS',
 'ADANIPOWER.NS',
 'BAJAJ-AUTO.NS',
 'HAL.NS',
 'LTIM.NS',
 'IOC.NS',
 'DLF.NS',
 'TATASTEEL.NS',
 'VBL.NS',
 'JIOFIN.NS',
 'SBILIFE.NS',
 'SIEMENS.NS',
 'GRASIM.NS',
 'HDFCLIFE.NS',
 'HINDALCO.NS',
 'PIDILITIND.NS',
 'BEL.NS',
 'HINDZINC.NS',
 'IRFC.NS',
 'BRITANNIA.NS',
 'PFC.NS',
 'INDUSINDBK.NS',
 'TECHM.NS',
 'BANKBARODA.NS',
 'ADANIENSOL.NS',
 'GODREJCP.NS',
 'INDIGO.NS',
 'EICHERMOT.NS',
 'RECLTD.NS',
 'ATGL.NS',
 'TRENT.NS',
 'ZOMATO.NS',
 'GAIL.NS',
 'TATAPOWER.NS',
 'CHO

In [50]:
#minute_data_1000 = yf.Tickers(yahoo_symbs[:1000]).history(start=start, interval ='1m')

In [51]:
all_yf_data = []
failed_tickers = []
for t in yahoo_symbs:
    try:
        yf_data = yf.download(t,start=start, interval ='1m')
        yf_data['ticker']=t
        all_yf_data.append(yf_data)
    except Exception as e :
        print(f"error while downloading for {t}")
        failed_tickers.append(t)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

In [52]:
minute_data= pd.concat(all_yf_data)

In [53]:
minute_data.head()

,Open,High,Low,Close,Adj Close,Volume,ticker
2024-04-01 09:15:00+05:30,2984.949951,2984.949951,2977.600098,2977.600098,2977.600098,0.0,RELIANCE.NS
2024-04-01 09:16:00+05:30,2977.850098,2981.649902,2977.850098,2981.649902,2981.649902,30310.0,RELIANCE.NS
2024-04-01 09:17:00+05:30,2981.649902,2982.399902,2976.600098,2979.949951,2979.949951,28908.0,RELIANCE.NS
2024-04-01 09:18:00+05:30,2980.449951,2987.949951,2980.300049,2985.050049,2985.050049,25789.0,RELIANCE.NS
2024-04-01 09:19:00+05:30,2984.250000,2986.399902,2982.550049,2986.250000,2986.250000,25454.0,RELIANCE.NS


In [54]:
minute_data.to_csv('minute_data_2.csv',compression='zip')

In [55]:
minute_data.shape[0

(1454856, 7)

In [71]:
set(minute_data.reset_index()['index'].apply(lambda x :x.strftime('%Y%m%d')))

{'20240401', '20240402', '20240403'}